# AI 웹 구조 네비게이터

## 환경 설정 및 API 키 로드

모든 라이브러리 가져옴 -> '.env'파일에 숨겨둔 제미나이 API 키를  가져온다. BeautifulSoup는 Html전처리, 'gtts'는 음성변환

In [ ]:
import google.generativeai as genai     # 제미나이 API 호출
from gtts import gTTS                   # Google Test - to - Speech
import os                               # 환견 변스 접근
from dotenv import load_dotenv          # .env파일 로드
from IPython.display import display, Audio, Markdown #데모 시연 표시
from bs4 import BeautifulSoup           # 전처리용

# API 키 설정
load_dotenv()
API_KEY = os.getenv('GEMINI_API_KEY')

if not API_KEY:
    # 키가 없는 경우 에러 메시지
    print( "API 키 파일을 찾을 수 없습니다")

else:
    try:
        # API 키로 제미나이 설정
        genai.configure(api_key=API_KEY)
        display(Markdown("API 키 및 라이브러리 설정 완료"))
    except Exception as e :
        print("API 키 설정 중 오류 발생")

print("--- 💡 내 API 키로 사용 가능한 모델 목록 ---")
for m in genai.list_models():
    if 'generateContent' in m.supported_generation_methods:
        print(m.name)
print("---------------------------------------")

## AI 모델 설정 및 핵심 프롬프트 함수 정의

create_navigation_prompt : AI에게 역할 부여 및 분석해야하는 구체적인 지시 정의

In [ ]:
# 제미나이 AI 모델 설정
try:
    # 설정한 모델 로드.
    model = genai.GenerativeModel('models/gemini-pro-latest')
    display(Markdown(f"{model.model_name} 로드 완료"))
except Exception as e :
    print(f"모델 불러오는 중 오류 발생, 오류 -> {e}")

# 프롬프트 엔지니어링
# 웹 접근성 전문가 역할 부여
# html을 분석해 '오디오 스크립트'를 생성하도록 지시.
def create_navigation_prompt(html_code):
    # 전처리된 HTML을 받습니다.
    return f"""
    당신은 20년 경력의 '웹 접근성 전문가'입니다.
    당신의 임무는 시각장애인 사용자를 위해, 이 복잡한 HTML 소스 코드를 분석하여 '오디오 네비게이션 가이드'를 생성하는 것입니다.

    [규칙]
    1.  당신은 전처리되어 '핵심'만 남은 HTML (nav, main 등)을 받았습니다.
    2.  `<nav>`, `<main>`, `<h1>`, `<h2>` 태그를 중심으로 분석하세요.

    [HTML 소스 코드]
    {html_code} # 전처리가 되었으니 3만자 한도도 제거

    [출력 포맷]
    위 HTML을 분석하여, 아래 4가지 질문에 대한 답변을 '오디오 스크립트' 형식으로만 생성해주세요. (그 외의 설명이나 인사말은 절대 넣지 마세요.)

    1.  페이지 제목 (Title): 이 페이지의 `<h1>` 또는 `<title>` 태그에 기반한 핵심 주제가 무엇인가요?
    2.  메인 메뉴 (Navigation): `<nav>` 태그 안의 주요 메뉴 항목(링크 텍스트)을 최대 5개까지 알려주세요.
    3.  핵심 본문 요약 (Main Summary): `<main>` 또는 `<article>` 태그 내부의 첫 번째 문단을 2~3문장으로 요약해주세요.
    """

## HTML 파일 전처리

'inputHtml.html' 파일에서 원본 읽어 옴
`BeautifulSoup`을 이용해 원본 HTML에서 불필요한 '노이즈' 태그를 모두 제거합니다.

In [ ]:
from bs4 import BeautifulSoup

input_file_path = 'inputHtml.html'  # 분석할 파일 이름
html_input = ""                     # 원본 HTML
html_cleaned = ""                   # 전처리한 HTML

try:
    # 1. 파일 읽기
    with open(input_file_path, 'r', encoding='utf-8') as f:
        html_input = f.read()

    if not html_input.strip():
        display(Markdown(f"{input_file_path} 파일이 비어있음"))
    else:
        display(Markdown(f"{input_file_path} 파일 읽기 성공(원본 크기 : {len(html_input)}"))
        display(Markdown(" 전처리 시작 ........"))

        # BeautifulSoup으로 HTML 파싱
        soup = BeautifulSoup(html_input, 'html.parser')

        # 불필요한 태그 제거
        for tag in soup(["script", "style", "footer", "aside", "form", "button"]):
            tag.decompose()

        # 필요한 태그만 남김
        # 만약 <main>태그가 있으면 사용, else-> <body>전체 쓴다
        main_content = soup.find("main")
        nav_content = soup.find("nav")
        # 꺠끗한 HTML 조각을 모을 리스트
        cleaned_parts = []

        # <nav> 태그 내용 추가
        if nav_content:
            cleaned_parts.append(str(nav_content))

        # <main> 태그 내용 추가
        if main_content:
            cleaned_parts.append(str(main_content))

        elif soup.body:
            # <main>이 없다면, <body>에서 불필요한 걸 뺀 나머지 사용
            cleaned_parts.append(str(soup.body))
        else:
            # <main>, <body> 둘 다 없으면 그냥 원본의 축소판을 씀
            cleaned_parts.append(soup.get_text(separator="\n", strip=True)[:5000])

        # 전처리 된 데이터를 'html_cleaned'에 저장.
        html_cleaned = "\n".join(cleaned_parts)

        display(Markdown(f"전처리 완료 (크기 : {len(html_cleaned)}"))

except FileNotFoundError:
    display(Markdown(f"{input_file_path}' 파일을 찾을 수 없습니다."))
except Exception as e:
    print(f"파일 읽기 또는 전처리 중 오류 발생 -> {e}")


## AI 호출 및 텍스트 결과 출력

In [ ]:
# 제미나이 API 호출
ai_script = ""
# 전처리 html 존재시만 AI 호출
if html_cleaned.strip():
    try:
        display(Markdown("제미나이가 HTML 구조 분석 중....."))
        # 전처리 데이터 담은 변수 -> 프롬프트 함수에 전달
        prompt = create_navigation_prompt(html_cleaned)
        response = model.generate_content(prompt)
        # 분석 결과
        ai_script = response.text
        display(Markdown("-----분석 결과--------"))
        display(Markdown(ai_script))

    except Exception as e :
        print(f"분석 중 오류 발생 -> {e}")
else:
    display(Markdown("현재 HTML 파일이 비어 있어 호출 실패."))

## TTS 변환 및 오디오 플레이어 출력

In [ ]:
# 음셩 변환 및 재생
tts_file_path = 'web_summary.mp3'

# ai가 생성한 텍스트 스크립트가 존재 시 만 실행
if ai_script:
    try:
        display(Markdown("HTML 분석 결과 음성으로 변환 중 ......"))

        # ai_script를 gTTS로 넘겨 mp3 파일로 저장.
        tts = gTTS(text=ai_script, lang='ko')
        tts.save(tts_file_path)

        display(Markdown("mp3파일 생성 완료!!!!"))

        #IPython.display.Audio를 사용해 노트북 셀에 바로 오디오 플레이어 띄움
        display(Audio(tts_file_path, autoplay=False))

    except Exception as e :
        print(f"음성 변환 중 오류 발생 -> {e}")
else :
    display(Markdown("AI 스크립트가 없으 음성 변환 실행 안함."))